Hello kagglers, I wanted to start this proyect to check if I can play a little bit with visualizations and visual insights. I wantted to practice a little bit of seaborn and matplotlib, maybe I'll be including some plotly.

We are dealing with Titanic (Reforged version or futuristic version).
Let's take a look at the features:
* PassengerId : includes a group or a sole person (group_number + _ + personal_id_number)
* HomePlanet : We have 3 "categories" Earth, Europe, Mars (it's gonna be interesting if at some point in time it's a planet discovered and we call it Europe)
* CryoSleep : Boolean (True or False) if the passenger was in suspended mode or not.
* Cabin : where the passanger was staying: it says that" the form deck/num/side, where side can be either P for Port or S for Starboard"
* Destination : Similar to HomePlanet, there are some categories to the "places" they were going.
* Age : self explained, Age for each passanger
* VIP : Boolean if each passanger whether or not pay for VIP
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck ---> Amenities 
* Name : Passangers name
* Transported : label or target column, whether or not the passanger was trasnported.

In [ ]:
#let's import some packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Let's check how are the values 

In [ ]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df.head(20)

In [ ]:
df.tail(20)

there are some missing values, here and there.
let's check the DF info adn check data types

In [ ]:
df.info()

There are some missing values in almost all the features.
Let's visualize Missing Values %

In [ ]:
(df.isnull().sum()/len(df)*100).sort_values()

In [ ]:
values_ = (df.isnull().sum()/len(df)*100).sort_values().values
values_

I want to plot the % of missing value

In [ ]:
(df.isnull().sum()/len(df)*100).sort_values().plot(kind = 'barh', figsize = (5,8), color = 'C1', fontsize = 15, xlim =(0, 3))
plt.title('% Missing Values', fontsize = 20 , color = 'red')
plt.xticks([0,1,2,3])
for i in range(len(values_)):
    plt.text(x=values_[i]+0.01,y = i-0.15, s=f'{round(values_[i],2)}%', color = f"C{i}", fontsize = 12)

I was considering removing all the missing values since each feature only has less than 2.5% of missing values.
and why not I was wondering.
But then I realize that if that's gonna happen I'm gonna get rid of the ~25% of the data which is catasthrophic in terms of getting a robust model predictions
Let's show it:


In [ ]:
total_of_removed_missing_values = 1 -  len(df.dropna(axis = 0)) / len(df)
print(f"Total percentage of missing values removed : {round(total_of_removed_missing_values *100,2)} %")

Therefore, removing the missing values this way, not gonna make it work, I imagine.
So we are going to do it the "hard"way let's take a closer look at each feature

In [ ]:
def mapper(df,cols, map_):
    
    for col in cols:
        df[col] = df[col].map(map_)
    return df

def fill_na_(df,cols, values):
    for col in cols:
        df[col] = df[col].fillna(values)
    return df

This mapper function will make booleans features to become numerical

In [ ]:
map_ = {
    True:1,
    False:0    
}
cols = ['VIP','CryoSleep','Transported']
df = mapper(df, cols, map_)
df

In [ ]:
cols = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

df = fill_na_(df,cols, 0)

In amenities columns we're gonna put all of the missing values to 0. It's not ok but it's better than putting any number, or maybe, I dont' know yet, but it seems to me a good idea to fill those missing values by groups and fill it with the median of each group, but that's tryhard code.

In [ ]:
cols = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
values = 0
for col in cols:
    values += df[col].values
df.loc[:, 'amen_total'] = values

Since we already filled all amenities MV: we gonna create a new big feature named amen_total : 
Total amenities which will be the sum of all the amenities.


Let's check out a plot of amen_total to check values

In [ ]:
plt.figure(figsize = (12,6))
plt.title("Amenities")
ax = sns.distplot(df['amen_total'], kde = True, color = 'C4')


A couple of interesting things:
* a lot of people doesn't have any amenities during the voyagem either because they don't have money
* a lot of people were actually Cryosleeping. so they could not spend in amenities

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.countplot(df[(df.amen_total > 0)]['CryoSleep'], color = 'g')

This is a nice plot, While people is Cryoslepping it's not possible at all to go have amenities for any of them.

In [ ]:
idxs = df.loc[~(df['CryoSleep']==1) & ~(df['CryoSleep']==0) & (df.amen_total>0)]['CryoSleep'].index

we are getting the indexes of the Cryosleep missing values and replacing all those who have amenities >0, have not been Cryoslept.

In [ ]:
df.loc[idxs,'CryoSleep'] = 0

In [ ]:
value_ = df.CryoSleep.value_counts()
value_

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize =(6,4))
ax = sns.countplot(df.CryoSleep, data = df);
plt.ylabel('Number of People', rotation = 0, labelpad=80)
plt.show()


* There are more people which haven't been CryoSlept.

In [ ]:
idx2 = df[df['CryoSleep'].isnull()].index

In [ ]:
df.loc[idx2,'CryoSleep'] = 1

I'm assuming that any other missing value here is just 1. Because they didn't have any amenities during the voyage. For instance, I can imagine they might be sleept.

In [ ]:
df[df.HomePlanet.isnull()].head()

In [ ]:
df['HomePlanet'].value_counts()

In [ ]:
df.Transported.value_counts()

In [ ]:
df.groupby('HomePlanet')['Transported'].count()

In [ ]:
plt.rcParams.update({'figure.autolayout': True})
plt.style.use('fivethirtyeight')


plt.figure(figsize = (10,7))
ax = sns.countplot(x = 'Transported', hue = 'HomePlanet', data = df);
ax.set_title('Home Planet Count',fontsize=20);
ax.set_ylabel('Nº.People', rotation=0, fontsize=20, labelpad=80)

labels = ax.get_yticklabels()
plt.setp(labels, rotation=45, horizontalalignment='right');

In [ ]:
plt.rcParams.update({'figure.autolayout': True})
plt.style.use('fivethirtyeight')


plt.figure(figsize = (10,7))
ax = sns.countplot(x = 'Transported', hue = 'CryoSleep', data = df);
ax.set_title('CryoSleep Count',fontsize=20);
ax.set_ylabel('Nº.People', rotation=0, fontsize=20, labelpad=80)

labels = ax.get_yticklabels()
plt.setp(labels, rotation=45, horizontalalignment='right');

In [ ]:
df.groupby(['Transported','VIP'])['VIP'].count()

In [ ]:
def give_me_quartiles(df,x,y):
    grouper = df.groupby(x)[y]
    min_,q_1,q_2,q_3,max_ = [grouper.quantile(number/4) for number in range(0,5)]
    return q_1,q_2,q_3,max_, min_, q_3 + (q_3-q_1)*1.5 


In [ ]:
df.loc[:,'amenities_bool'] =np.where(values> 0, 1,0)
df

In [ ]:

plt.figure(figsize = (10,5))
ax = sns.countplot(x='amenities_bool', hue ='Transported', data = df)
for _,p in enumerate(ax.patches):
    ax.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center',
                va = 'center', xytext = (0, 10),
                textcoords = 'offset points')
                
#     ax.text(x = p.get_x()+0.42, 
#             y = (p.get_height()/2)+1,
#             s = f'{list_[_]}%',
#             color =f'C{7-_}'
#            )


In [ ]:
df['PassengerId'].apply(lambda x : int(str(x).split('_')[0])).nunique()

In [ ]:
l_ = df['PassengerId'].apply(lambda x : int(str(x).split('_')[0])).values
l2 = [1 if list(l_).count(valor)>=2 else 0 for valor in l_]

df.loc[:,'with_sg']= np.array(l2)

In [ ]:
sns.countplot(x = 'with_sg', hue = 'Transported', data = df)

In [ ]:
x = df.loc[df['Age']<=15]['amenities_bool']
sns.countplot(x, hue ='Transported', data = df)

In [ ]:
sns.countplot(x = 'with_sg', data = df)

In [ ]:
df['groups'] = df['PassengerId'].apply(lambda x: int(str(x).split('_')[0]))

In [ ]:
plt.figure(figsize = (10,7))
ax = sns.histplot(df[(df['amen_total']> 0) & (df['with_sg']==1)].groupby('groups')['amen_total'].sum(), bins=20, color ='C1')

In [ ]:
x = df[(df['amen_total']> 0) & (df['with_sg']==1)].groupby('groups')['amen_total'].sum()
x.sort_values(ascending = False)

In [ ]:
df.loc[df['groups']==4594]

In [ ]:
df[df['with_sg']==1].groupby('groups')['Transported'].count().sort_values()

In [ ]:
df[df['with_sg']==0].groupby('groups')['Transported'].count().sort_values()

In [ ]:
df.loc[df['groups']==8988]

In [ ]:
sns.barplot(x ='with_sg', y='amen_total', data = df)

In [ ]:
A = df.loc[df.with_sg == 1]['Cabin'].isnull()
# l_ = [index for (index,element) in enumerate(A.isnull()) if element]
# df.iloc[l_]    
df.iloc[A[A==True].index]

In [ ]:
list_ = df.groupby(['Transported', 'VIP'])['VIP'].count()
list_.loc[0,:] = round(list_.loc[0,:] / sum(list_.loc[0,:]) *100,2)
list_.loc[1,:] = round(list_.loc[1,:] / sum(list_.loc[1,:]) *100,2)
list_ = sorted(list_.values,reverse =True)

In [ ]:
plt.rcParams.update({'figure.autolayout': True})
plt.style.use('fivethirtyeight')


plt.figure(figsize = (10,7))
ax = sns.countplot(x = 'Transported',
                   hue = 'VIP',
                   data = df);
ax.set_title('VIP Count',
             fontsize=20);

ax.set_ylabel('Nº.People',
              rotation=0,
              fontsize=20,
              labelpad=80)
labels = ax.get_yticklabels()
plt.setp(
    labels,
    rotation=45,
    horizontalalignment='right'
);

for _,p in enumerate(ax.patches):
    ax.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center',
                va = 'center', xytext = (0, 10),
                textcoords = 'offset points')
                
    ax.text(x = p.get_x()+0.42, 
            y = (p.get_height()/2)+1,
            s = f'{list_[_]}%',
            color =f'C{7-_}'
           )

In [ ]:
plt.rcParams.update({'figure.autolayout': True})
plt.style.use('fivethirtyeight')
plt.figure(figsize = (10,7))
ax = sns.boxplot(x = df['Transported'],y = df['Age'] );
ax.set_title('Age Count',fontsize=20);
ax.set_ylabel('Age', rotation=0, fontsize=20, labelpad=20)
labels = ax.get_yticklabels()
plt.setp(labels, rotation=45, horizontalalignment='right');
Q_l = give_me_quartiles(df, 'Transported', 'Age')
for Q in Q_l:
    for i in range(len(Q)):
        ax.annotate(str(Q[i]), xy =(i-0.056, Q[i]+1), horizontalalignment='center')